In [1]:
using Revise, HarmonicBalance

  Activating project at `~/.julia/environments/v1.7`


# Define the differential equation of motion

In [2]:
@variables α, ω, ω0, F, t, η, Δω, x(t);

In [30]:
diff_eq = DifferentialEquation(d(x,t,2) + ω0^2*x + α*x^3 ~ F*cos(ω*t), x)
add_harmonic!(diff_eq, x, ω) # specify the ansatz x = u(T) cos(ωt) + v(T) sin(ωt)
add_Hopf!(diff_eq, Δω)

# implement ansatz to get harmonic equations
harmonic_eq = get_harmonic_equations(diff_eq)


A set of 6 harmonic equations
Variables: u1(T), v1(T), u2(T), v2(T), u3(T), v3(T)
Parameters: α, ω, Δω, ω0, F

Harmonic ansatz: 
x(t) = u1(T)*cos(ωt) + v1(T)*sin(ωt) + u2(T)*cos(Δω + ωt) + v2(T)*sin(Δω + ωt) + u3(T)*cos(ω - Δωt) + v3(T)*sin(ω - Δωt)

Harmonic equations:

(ω0^2)*u1(T) + (3//4)*α*(u1(T)^3) + (2//1)*ω*Differential(T)(v1(T)) + (3//2)*α*(u2(T)^2)*u1(T) + (3//2)*α*(u3(T)^2)*u1(T) + (3//4)*α*(v1(T)^2)*u1(T) + (3//2)*α*(v2(T)^2)*u1(T) + (3//2)*α*(v3(T)^2)*u1(T) + (3//2)*α*u1(T)*u2(T)*u3(T) + (3//2)*α*u2(T)*v1(T)*v3(T) + (3//2)*α*u3(T)*v1(T)*v2(T) - (ω^2)*u1(T) - (3//2)*α*u1(T)*v2(T)*v3(T) ~ F

(ω0^2)*v1(T) + (3//4)*α*(v1(T)^3) + (3//4)*α*(u1(T)^2)*v1(T) + (3//2)*α*(u2(T)^2)*v1(T) + (3//2)*α*(u3(T)^2)*v1(T) + (3//2)*α*(v2(T)^2)*v1(T) + (3//2)*α*(v3(T)^2)*v1(T) + (3//2)*α*u1(T)*u2(T)*v3(T) + (3//2)*α*u1(T)*u3(T)*v2(T) + (3//2)*α*v1(T)*v2(T)*v3(T) - (ω^2)*v1(T) - (2//1)*ω*Differential(T)(u1(T)) - (3//2)*α*u2(T)*u3(T)*v1(T) ~ 0

(ω0^2)*u2(T) + (2//1)*Δω*Differential(T)(v2(T)) + (3

# Hopf manipulation

In [22]:
# remove one Hopf variable, replace by Δω
prob = HarmonicBalance.Hopf._Hopf_Problem(harmonic_eq, Δω);

# show the current variables
#get_variables(harmonic_eq)

In [23]:
using HarmonicBalance.DataStructures
var_soln = Dict([v => randn() for v in HarmonicBalance._remove_brackets.(harmonic_eq.variables)])
dum_soln = merge(Dict(α => 1.0, F => 0.01, ω => 1.0, ω0 => 1.05), var_soln)
prob.jacobian(OrderedDict(dum_soln))

5×5 Matrix{ComplexF64}:
  -0.52216+0.0im    2.31908+0.0im  …   0.889323+0.0im  -0.964784+0.0im
  -3.93084+0.0im    0.52216+0.0im       2.13917+0.0im   -1.83923+0.0im
   -2.7188+0.0im    1.13029+0.0im       4.63815+0.0im   -1.46846+0.0im
 -0.935889+0.0im   0.104088+0.0im     -0.113705+0.0im  -0.918291+0.0im
  0.900512+0.0im  -0.176012+0.0im     -0.732977+0.0im        0.0+0.0im

In [27]:
prob.eom.equations

6-element Vector{Symbolics.Equation}:
 (ω0^2)*u1(T) + (3//4)*α*(u1(T)^3) + (2//1)*ω*Differential(T)(v1(T)) + (3//4)*α*(v1(T)^2)*u1(T) + (3//2)*α*(u2(T)^2)*u1(T) + (3//2)*α*(u3(T)^2)*u1(T) + (3//2)*α*(v2(T)^2)*u1(T) + (3//2)*α*u1(T)*u2(T)*u3(T) + (3//2)*α*u3(T)*v1(T)*v2(T) - (ω^2)*u1(T) ~ F
 (ω0^2)*v1(T) + (3//4)*α*(v1(T)^3) + (3//4)*α*(u1(T)^2)*v1(T) + (3//2)*α*(u2(T)^2)*v1(T) + (3//2)*α*(u3(T)^2)*v1(T) + (3//2)*α*(v2(T)^2)*v1(T) + (3//2)*α*u1(T)*u3(T)*v2(T) - (ω^2)*v1(T) - (2//1)*ω*Differential(T)(u1(T)) - (3//2)*α*u2(T)*u3(T)*v1(T) ~ 0
 (ω0^2)*u2(T) + (3//4)*α*(u2(T)^3) + (2//1)*ω*Differential(T)(v2(T)) + (2//1)*Δω(T)*Differential(T)(v2(T)) + (3//2)*α*(u1(T)^2)*u2(T) + (3//4)*α*(u1(T)^2)*u3(T) + (3//4)*α*(v2(T)^2)*u2(T) + (3//2)*α*(u3(T)^2)*u2(T) + (3//2)*α*(v1(T)^2)*u2(T) - (ω^2)*u2(T) - (Δω(T)^2)*u2(T) - (3//4)*α*(v1(T)^2)*u3(T) - (2//1)*ω*u2(T)*Δω(T) ~ 0
 (ω0^2)*v2(T) + (3//4)*α*(v2(T)^3) + (3//4)*α*(u2(T)^2)*v2(T) + (3//2)*α*(u1(T)^2)*v2(T) + (3//2)*α*(u3(T)^2)*v2(T) + (3//2)*α*(

# Solving the resulting polynomial equations

In [29]:
fixed = (α => 1., ω0 => 1.0, F => 0.01)   # fixed parameters
swept = ω => LinRange(1.1, 1.2, 3)           # range of parameter values
solutions = get_steady_states(prob, swept, fixed, random_warmup=false)

Solving via total degree homotopy ... 100%|██████████████████████████████████████████████████| Time: 0:00:30


LoadError: No solutions found!

In [ ]:
@doc HarmonicBalance.HomotopyContinuation.newton

In [48]:
using LinearAlgebra
pars = [1.0, 1.1, 1.0, 0.01]

code = :rejected
while code == :rejected
    res = HarmonicBalance.HomotopyContinuation.newton(prob.system, cat(-0.5*randn(2), -0.001*randn(3), 1,dims=1), pars, max_iters=300, atol=1e-10)
    code = res.return_code
end

norm(res.x).^2
res.return_code

:success

In [49]:
res.x

6-element Vector{ComplexF64}:
    -0.5035067606122274 + 0.0im
 -8.546010971183819e-65 + 0.0im
 -4.527839539413356e-72 + 0.0im
  2.263919769706678e-72 + 0.0im
                    0.0 + 0.0im
     1.1410232244247949 + 0.0im

In [ ]:
plt = plot_1D_solutions(solutions, x="F0", y="u1^2 + v1^2");
#HarmonicBalance.savefig("asym.png")

In [ ]:
import HarmonicBalance.TimeEvolution: ODEProblem, DifferentialEquations.solve, ParameterSweep
initial_state = solutions[1][1]

T = 2E6
sweep = ParameterSweep(F0 => (0.002, 0.011), (0,T))
TDproblem = ODEProblem(harmonic_eq, initial_state, sweep=sweep, timespan=(0,2*T))
TDsoln = solve(TDproblem, saveat=1);

In [ ]:
plot(TDsoln.t, getindex.(TDsoln.u, 1));
#HarmonicBalance.xlabel("time", fontsize=24)
#HarmonicBalance.ylabel(HarmonicBalance.latexify("u_1"), fontsize=24)

In [ ]:
u(i) = getindex.(TDsoln.u, i)[Int(1.5*T):end]
plot(u(1), u(2))
plot(u(3), u(4));
#HarmonicBalance.xlabel(HarmonicBalance.latexify("u_1, u_2"), fontsize=24)
#HarmonicBalance.ylabel(HarmonicBalance.latexify("v_1, v_2"), fontsize=24)